## Membuat Virtual Environment

In [1]:
!pip install uv
!uv pip install virtualenv
!virtualenv hailoworld --py=3.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.2/21.2 MB 77.2 MB/s eta 0:00:00
Using Python 3.12.11 environment at: /usr
Resolved 4 packages in 137ms
Prepared 2 packages in 127ms
Installed 2 packages in 6ms
 + distlib==0.4.0
 + virtualenv==20.34.0
created virtual environment CPython3.10.12.final.0-64 in 1056ms
  creator CPython3Posix(dest=/content/hailoworld, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.2, setuptools==80.9.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


## Download Hailo Compiler

In [2]:
!gdown "1jopeKVCSCTWCSLTH5kfNtQvbp2EjJfEc"
!gdown "1H8VBpLBGw2JeXRHk5sthBe-TYjMmfuW-"
!gdown "1qDAssC3Qbzi9EuGJP4TpVzV5p1EYTfqU"

Downloading...
From (original): https://drive.google.com/uc?id=1jopeKVCSCTWCSLTH5kfNtQvbp2EjJfEc
From (redirected): https://drive.google.com/uc?id=1jopeKVCSCTWCSLTH5kfNtQvbp2EjJfEc&confirm=t&uuid=3dcec474-9ec4-4ef8-8afd-ba5739624f9b
To: /content/hailo_dataflow_compiler-3.33.0-py3-none-linux_x86_64.whl
100% 512M/512M [00:10<00:00, 50.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1H8VBpLBGw2JeXRHk5sthBe-TYjMmfuW-
To: /content/hailort-4.23.0-cp310-cp310-linux_x86_64.whl
100% 9.98M/9.98M [00:00<00:00, 50.8MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1qDAssC3Qbzi9EuGJP4TpVzV5p1EYTfqU
From (redirected): https://drive.google.com/uc?id=1qDAssC3Qbzi9EuGJP4TpVzV5p1EYTfqU&confirm=t&uuid=3366992d-e1f3-48d7-bde0-9a84953f52e1
To: /content/hailort_4.23.0_amd64.deb
100% 7.60M/7.60M [00:00<00:00, 46.9MB/s]


## Terminal Execution (Linux)

In [ ]:
# source hailoworld/bin/activate
# sudo apt-get install -y graphviz graphviz-dev pkg-config
# sudo dpkg -i ./hailort_4.23.0_amd64.deb
# pip install ./hailort-4.23.0-cp310-cp310-linux_x86_64.whl
# pip install ./hailo_dataflow_compiler-3.33.0-py3-none-linux_x86_64.whl
# git clone https://github.com/hailo-ai/hailo_model_zoo
# cd hailo_model_zoo
# git checkout -b update-to-version-2.17 origin/update-to-version-2.17
# pip install -e . --use-pep517 --config-settings editable_mode=compat --no-build-isolation
# cd ..
# git clone https://github.com/maybeitsai/convert-model-to-hailo.git

## Cek model

In [ ]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 108.1 MB/s eta 0:00:00


In [ ]:
import onnx

model = onnx.load("/content/convert-model-to-hailo/models/onnx/face-expression.onnx")
print("Inputs:", [i.name for i in model.graph.input]) # ambil nama array ny dari sini buat masukin ke flag `--start/end-node-names`
print("Outputs:", [o.name for o in model.graph.output])  # dan ini

Inputs: ['input']
Outputs: ['output']


## Convert to .har

In [ ]:
# USE GPU
# hailo parser onnx /content/convert-model-to-hailo/models/onnx/face-expression.onnx \
#   --hw-arch hailo8 \
#   --start-node-names input \
#   --end-node-names output

# mkdir -p convert-model-to-hailo/models/har && \
# mv face-expression.har convert-model-to-hailo/models/har/

# hailo parser onnx /content/convert-model-to-hailo/models/onnx/pose-recognition.onnx \
#   --hw-arch hailo8 \
#   --start-node-names input \
#   --end-node-names output

# mkdir -p convert-model-to-hailo/models/har && \
# mv pose-recognition.har convert-model-to-hailo/models/har/


# USE CPU
# CUDA_VISIBLE_DEVICES=-1 hailo parser onnx /content/convert-model-to-hailo/models/onnx/face-expression.onnx \
#   --hw-arch hailo8 \
#   --start-node-names input \
#   --end-node-names output

# CUDA_VISIBLE_DEVICES=-1 hailo parser onnx /content/convert-model-to-hailo/models/onnx/pose-recognition.onnx \
#   --hw-arch hailo8 \
#   --start-node-names input \
#   --end-node-names output

# mkdir -p convert-model-to-hailo/models/har && \
# mv pose-recognition.har convert-model-to-hailo/models/har/


## Optimisasi format .har biar bisa di konvert ke .hef

sebenarnya disini bisa pake flag --calib-path /path/gambar/lu untuk nge gantiin si --use-random-calib-set, klo pengen dataset RAW (gosah di labelin) test sendiri dengan split 50/50 buat setiap class yang ada. Ngaada aturan minimal gambar yang diperluuin.

In [ ]:
# # CALIBRATE
# hailo optimize /content/convert-model-to-hailo/models/har/face-expression.har \
#   --hw-arch hailo8 \
#   --calib-set-path /content/convert-model-to-hailo/data/calibration/face_expression \
#   --output /content/convert-model-to-hailo/models/har/face-expression-opt.har

# hailo optimize /content/convert-model-to-hailo/models/har/pose-recognition.har \
#   --hw-arch hailo8 \
#   --calib-set-path /content/convert-model-to-hailo/data/calibration/pose_recognition \
#   --output /content/convert-model-to-hailo/models/har/pose-recognition-opt.har

# # UNCALIBRATE
# hailo optimize /content/convert-model-to-hailo/models/har/face-expression.har \
#   --hw-arch hailo8 \
#   --use-random-calib-set \
#   --output /content/convert-model-to-hailo/models/har/face-expression-opt.har

# hailo optimize /content/convert-model-to-hailo/models/har/pose-recognition.har \
#   --hw-arch hailo8 \
#   --use-random-calib-set \
#   --output /content/convert-model-to-hailo/models/har/pose-recognition-opt.har

In [ ]:
# mkdir -p convert-model-to-hailo/models/hef/face-expression && \
# mkdir -p convert-model-to-hailo/models/hef/pose-recognition && \

# hailo compiler /content/convert-model-to-hailo/models/har/face-expression-opt.har \
#   --hw-arch hailo8 \
#   --output-dir /content/convert-model-to-hailo/models/hef/face-expression/

# hailo compiler /content/convert-model-to-hailo/models/har/pose-recognition-opt.har \
#   --hw-arch hailo8 \
#   --output-dir /content/convert-model-to-hailo/models/hef/pose-recognition/

In [5]:
!zip -r har_models.zip /content/convert-model-to-hailo/models/har/
!zip -r hef_models.zip /content/convert-model-to-hailo/models/hef/

updating: content/convert-model-to-hailo/models/har/ (stored 0%)
updating: content/convert-model-to-hailo/models/har/pose-recognition-opt.har (deflated 46%)
updating: content/convert-model-to-hailo/models/har/.ipynb_checkpoints/ (stored 0%)
updating: content/convert-model-to-hailo/models/har/face-expression.har (deflated 9%)
updating: content/convert-model-to-hailo/models/har/face-expression-opt.har (deflated 42%)
updating: content/convert-model-to-hailo/models/har/pose-recognition.har (deflated 9%)
  adding: content/convert-model-to-hailo/models/hef/ (stored 0%)
  adding: content/convert-model-to-hailo/models/hef/face-expression/ (stored 0%)
  adding: content/convert-model-to-hailo/models/hef/face-expression/hailo_sdk.core.log (deflated 94%)
  adding: content/convert-model-to-hailo/models/hef/face-expression/allocator.log (stored 0%)
  adding: content/convert-model-to-hailo/models/hef/face-expression/face-expression.hef (deflated 29%)
  adding: content/convert-model-to-hailo/models/he